In [70]:
import os
import requests 
import tiktoken
from groq import Groq
from tqdm import tqdm
from elasticsearch import Elasticsearch

# Fetching Data

In [71]:
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)
len(documents)

948

# Q2. Indexing the data

In [72]:
es_client = Elasticsearch('http://localhost:9200')

In [73]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} # primary key in RDBMS taste
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [74]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████| 948/948 [00:29<00:00, 32.53it/s]


In [78]:
def filter_search(query, size=5, search_words=[]):
    def filter_builder():
        return list(map(lambda word: {"term": {**word}}, search_words))
        
    search_query = {
        "size": size,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": filter_builder()
            }
        }
    }

    return es_client.search(index=index_name, body=search_query)

In [79]:
query = "How do I execute a command in a running docker container?"

# Q3. Searching

In [80]:
filter_search(query)["hits"]["max_score"]

84.050095

# Q4. Filtering

In [81]:
returned_docs = filter_search(query, 3, [{"course": "machine-learning-zoomcamp"}])["hits"]["hits"]
for doc in returned_docs: 
  print(doc["_source"]["question"])

How do I debug a docker container?
How do I copy files from my local machine to docker container?
How do I copy files from a different folder into docker container’s working directory?


In [82]:
returned_docs[2]["_source"]["question"]

'How do I copy files from a different folder into docker container’s working directory?'

# Q5. Building a prompt

In [83]:
context_template = """
Q: {question}
A: {text}
""".strip()

In [84]:
context = ""
for doc in returned_docs:
  context += context_template.format(question=doc["_source"]["question"], text=doc["_source"]["text"]) + "\n\n"

print(context)

Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/container
Hrithik Kumar Advani

Q: How do I copy files from a different folder into docker container’s working directory?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to 

In [85]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [86]:
prompt = prompt_template.format(question=query, context=context).strip()
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do I execute a command in a running docker container?

CONTEXT:
Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:

In [87]:
len(prompt)

1462

# Q6. Tokens

In [17]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [67]:
encoded_prompt = encoding.encode(prompt)
print(len(encoded_prompt))

322


# Bonus: generating the answer (ungraded)

In [33]:
def ask_groq(prompt, model="mixtral-8x7b-32768"):
    client = Groq(
        api_key=os.environ.get("GROQ_API_KEY"),
    )
    
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model=model,
    )

    return chat_completion.choices[0].message.content

In [34]:
answer = ask_groq(prompt)
print(answer)

To execute a command in a running docker container, you can use the `docker exec` command. The syntax is as follows:

```
docker exec -it <container-id> bash
```

You can find the container-id by running the `docker ps` command. Then replace `<container-id>` with your actual container id.

This command will open an interactive shell within the running container, allowing you to execute commands within the context of the container.


# Bonus: Bonus: calculating the costs (ungraded)

> As I used llama model, my answers won't be based on my solution 

In [64]:
def print_calculate_price(input_tokens: int, output_tokens: int, number_of_api_calls):
  input_price = 0.005 * input_tokens * number_of_api_calls / 1e3
  output_price = 0.015 * output_tokens * number_of_api_calls / 1e3
  print(f'total price = {input_price + output_price}$')

In [66]:
# check answers using: https://livechatai.com/gpt-4o-pricing-calculator
given_average_input_tokens_count = 150
given_average_output_tokens_count = 250
requests_number = 1000
print_calculate_price(given_average_input_tokens_count, given_average_output_tokens_count, requests_number)

total price = 4.5$


In [68]:
# using llama tokens count
print_calculate_price(len(encoded_prompt), len(encoding.encode(answer)), requests_number)

total price = 2.975$


# Cleaning up

In [69]:
es_client.options(ignore_status=[400,404]).indices.delete(index='course-questions')

ObjectApiResponse({'acknowledged': True})